In [ ]:
%pip install okareo attrs dateutils

In [ ]:
# perform a test run using a scenario set loaded in the previous cell 
import os
import random
import string
from okareo import Okareo
from okareo.model_under_test import ChromaDb, CohereModel, OpenAIModel
from okareo_api_client.models import TestRunType

# generate a random string to use in the test run name
rnd = "".join(random.choices(string.ascii_lowercase, k=5))

# API Key for Okareo runs
OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
COHERE_API_KEY = os.environ["COHERE_API_KEY"]

okareo = Okareo(OKAREO_API_KEY)
scenario = okareo.upload_scenario_set(file_path='./webbizz_10_articles.jsonl', 
                                      scenario_name="RAG with Webbizz articles")

# register both models
model_under_test_cohere = okareo.register_model(
        name=f"rag_cohere_chromadb",
        model=[
            ChromaDb(
                collection_name="test-collection-cohere",
            ),
            CohereModel(
                model_id="embed-english-light-v3.0",
                model_type="embed",
                input_type="search_query",
            ),
        ],
    )

model_under_test_openai = okareo.register_model(
     name=f"rag_openai_chromadb",
        model=[
            ChromaDb(
                collection_name="test-collection-openai",
            ),
            OpenAIModel(
                model_id="text-embedding-ada-002",
                model_type="embed",
            ),
        ],
    )

# fetch metrics for both the models
result_cohere = model_under_test_cohere.run_test_v2(
    name=f"rag-cohere-chromadb-{rnd}",
    scenario=scenario,
    calculate_metrics=True,
    test_run_type=TestRunType.INFORMATION_RETRIEVAL,
    api_keys={
        "chromadb": "",
        "cohere": os.getenv("COHERE_API_KEY"),
    },
)

result_openai = model_under_test_openai.run_test_v2(
    name=f"rag-openai-chromadb-{rnd}",
    scenario=scenario,
    calculate_metrics=True,
    test_run_type=TestRunType.INFORMATION_RETRIEVAL,
    api_keys={
        "chromadb": "",
        "openai": os.getenv("OPENAI_API_KEY"),
    },
)

# display model level metrics for the test run
print(result_openai.model_metrics.to_dict())
print(result_cohere.model_metrics.to_dict())